In [46]:
import urllib.request, os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import shutil
import os

In [47]:
data = pd.read_csv("PohitabelEN.csv")
data.head()

,Kohtuasja_ID,Kohtuasja_nr,Lõigu sisu,Pealkiri1,Tag
0,1,In Case C‑414/20 PPU,JUDGMENT OF THE COURT (Third Chamber),Koda,C19Centre
1,1,In Case C‑414/20 PPU,13 January 2021,Kuupaev,C19Centre
2,1,In Case C‑414/20 PPU,(Reference for a preliminary ruling – Urgent p...,Viited,C71Indicateur
3,1,In Case C‑414/20 PPU,"MM,",Poolte_nimed,C02AlineaAltA
4,1,In Case C‑414/20 PPU,"Spetsializirana prokuratura,",Poolte_nimed,C02AlineaAltA


In [48]:
column_list=data['Lõigu sisu'].to_list()
column_list[0]

'JUDGMENT OF THE COURT (Third Chamber)'

In [87]:
column_list_low=[str(x).lower() for x in column_list]   
column_list[0]

'JUDGMENT OF THE COURT (Third Chamber)'

In [113]:
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        for x in column_list_low:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(x, min_len=3)
            

In [114]:
import gensim.models

sentences = MyCorpus() #lõhub lause tükkideks
model = gensim.models.Word2Vec(sentences=sentences) #treenib mudeli ette antud andmete peal

In [111]:
next(iter(sentences))

['judgment', 'court', 'third', 'chamber']

In [110]:
#Proovitud salvestada mudelit, kus asub?
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
#to load tempfile:  new_model = gensim.models.Word2Vec.load(temporary_filepath)

In [115]:
model.most_similar('tax') 

<ipython-input-115-b78372aa2fdb>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('tax')


[('vat', 0.692110538482666),
 ('taxes', 0.6870538592338562),
 ('taxation', 0.6827908754348755),
 ('irc', 0.6739990711212158),
 ('fiscal', 0.6178154945373535),
 ('att', 0.5937644839286804),
 ('taxing', 0.5670206546783447),
 ('dividend', 0.5596171617507935),
 ('fid', 0.5499796867370605),
 ('dividends', 0.5380105972290039)]

In [116]:
# funktsioon pikemate lausevektorite loomiseks (mean)
def get_sum_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.wv.vocab]
    if len(words) >= 1:
        return np.sum(word2vec_model[words], axis=0)
    else:
        return []

In [117]:
# Käib kogu mudeli üle, selleks, et saada igast lõigust vektorid kasutades eelnevat funktsiooni get_mean_vector
sonad_arrays=[]
doc_jarjekord = []
jarjekorra_nr = []
num=0
for doc in sentences:
    vec = get_sum_vector(model, doc)
    if len(vec) > 0:
        sonad_arrays.append(vec)
        doc_jarjekord.append(doc)
        jarjekorra_nr.append(num)
    num+=1

<ipython-input-116-cdbb58556c9c>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.sum(word2vec_model[words], axis=0)


In [118]:
#loodud proovilause, millega testida sarnasusi
listike='Value added tax is important, tinkivinki.'

In [119]:
#teeb ette antud üheelemendilise listi vektoriks
#proovimudel nt get_mean_vector, get_sum_vector
def proovilistvektoriks(proovimudel, model, listike):
    proovivektor=proovimudel(model, utils.simple_preprocess(listike))
    return proovivektor

proovivektoriks1=proovilistvektoriks(get_mean_vector, model, listike)

<ipython-input-34-57fb566a9efa>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.mean(word2vec_model[words], axis=0)


In [123]:
def vordlus (sonad_arrays, jarjekorra_nr, proovivektor, column_list_low):
    sarnased_maatriks=cosine_similarity(np.stack(sonad_arrays, axis=0), Y=proovivektor.reshape(1, -1), dense_output=True)
    sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
    koige_sarnasem_idx=jarjekorra_nr[sorteeritud_indeksid[len(sorteeritud_indeksid) - 10]]
    return column_list_low[koige_sarnasem_idx]

vordlus(sonad_arrays, jarjekorra_nr, proovivektoriks1, column_list_low)

'–        as denominator, the total amount, exclusive of value added tax, of turnover per year attributable to transactions included in the numerator and to transactions in respect of which value added tax is not deductible. ...'

In [57]:
#rakendame koossiinuse sarnasuse leidmist (võrdleme prooviks ette antud lause vektorit kogu vektori peal)
#sarnased_maatriks=cosine_similarity(np.stack(sonad_arrays, axis=0), Y=proovivektoriks1.reshape(1, -1), dense_output=True)

In [58]:
# np.squeezega läheme maatriksi sissepoole ja argsort sorteerime tulemused ära
#sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
#sorteeritud_indeksid

array([ 41676, 108888, 241708, ..., 255002, 521009, 567465], dtype=int64)

In [66]:
#võtame sealt kõige viimase
#sarnased_maatriks[567465] #leiab koossiinuse sarnasuse

array([0.81828535], dtype=float32)

In [67]:
#jarjekorra_nr[567465] #leiab sarnase teksti õige indeksi

575959

In [68]:
#column_list_low[575959] #kasutades eelnevat indeksit leiab teksti

'–        value added tax,'